## Prepare

In [1]:
import torch, struct, os, psutil, subprocess, time, threading
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tensorflow as tf

from torch.utils.data import DataLoader, TensorDataset
import concurrent.futures
import pandas as pd

2024-02-04 00:17:56.307424: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-04 00:17:56.340109: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 00:17:56.340138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 00:17:56.340803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 00:17:56.345452: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Convert to PyTorch tensors
test_images_pt = torch.tensor(test_images).float()
test_labels_pt = torch.tensor(test_labels)
# Flatten and normalize the images
test_images_pt = test_images_pt.view(-1, 28*28) / 255.0  # Flatten and normalize

# Assuming test_images_pt is your PyTorch tensor with shape [num_samples, 784]
test_images_pt_reshaped = test_images_pt.view(-1, 1, 28, 28)  # Reshape to [num_samples, channels, height, width]

# Downsample images
test_images_pt_downsampled = F.interpolate(test_images_pt_reshaped, size=(14, 14), mode='bilinear', align_corners=False)

# Flatten the images back to [num_samples, 14*14]
test_images_pt_downsampled = test_images_pt_downsampled.view(-1, 14*14)

In [13]:
def evaluate_pytorch_model(model, datasets, labels):
    # Create TensorDataset for test data
    test_dataset = TensorDataset(datasets, labels)
    # Create a DataLoader for the test dataset
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [4]:
def load_img_from_file(data_file="input", show=False):
    try:
        with open(data_file, 'rb') as file:
            buf = file.read()
    except Exception as e:
        print(e)
        return None, e

    digits = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    l = int(np.sqrt(len(digits)))
    if show:
        c = ""
        for row in range(l):
            for col in range(l):
                if buf[row * l + col] > 230:
                    c += "&"
                else:
                    c += "-"
            c += "\n"
        print(c)

    return digits, None

def save_img_to_file(image, data_file = "input"):
    try:
        # Convert to bytes
        image_bytes = np.array(image*255).astype('uint8').tobytes()
        
        # Write to file
        with open(data_file, 'wb') as file:
            file.write(image_bytes)
        
        #print(f"Image saved to {data_file}")
    except Exception as e:
        print(f"Error saving image: {e}")

In [5]:
def monitor_memory(pid, freq = 0.01):
    p = psutil.Process(pid)
    max_memory = 0
    while True:
        try:
            mem = p.memory_info().rss / (1024 * 1024)
            max_memory = max(max_memory, mem)
        except psutil.NoSuchProcess:
            break  # Process has finished
        time.sleep(freq)  # Poll every second
        
    #print(f"Maximum memory used: {max_memory} MB")
    return max_memory

In [68]:
def benchmark(test_images, predictions, tmp_folder, model_in_path, vm_file = "./bin/vm", program = "./bin/mlgo_784_IDD.bin", threaded = True):
    #print (vm_file, program)
    
    benchmark_start_time = time.time()
    loss = 0
    mem_usage = []
    time_cost = []
    for ind, img in enumerate(test_images):
        img_out_path = tmp_folder + str(ind)
        save_img_to_file(img, img_out_path)

        # Exclusion of Pre-processing
        start_time = time.time()
        command = [f"{vm_file}", f"--basedir={tmp_folder}",
                f"--program={program}", f"--model={model_in_path}", 
                f"--data={img_out_path}", "--mipsVMCompatible"]
        
        print ("Process for image", ind)
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Get the process ID
        pid = process.pid

        if threaded:
            with concurrent.futures.ThreadPoolExecutor() as executor:
                future = executor.submit(monitor_memory, pid)
                _, stderr = process.communicate()
                max_memory = future.result()
                #print(f"Maximum memory used in multi-threaded mode: {max_memory} bytes")
        else:
            max_memory = monitor_memory(pid)  # Run in the same thread
            _, stderr = process.communicate()
            #print(f"Maximum memory used in single-threaded mode: {max_memory} bytes")
        pred = int(stderr[-2])
        if pred != predictions[ind]:
            print ("Loss on index", ind)
            loss += 1

        mem_usage.append(max_memory)
        time_cost.append(time.time() - start_time)
    
    print ("Total time:", time.time() - benchmark_start_time)
    #print ("total mem:", sum(mem))
    return loss, mem_usage, time_cost

def calculate_loss(veri_infer, predicted_labels):
    count = 0
    for i in range(len(veri_infer)):
        if veri_infer[i] != predicted_labels[i]:
            count +=1
            print (f"Index {i} Not match!")

    return count/len(veri_infer)*100

In [7]:
csv_path = '../../benchmarks/benchmark_results.csv'

columns = ['Framework', 'Architecture', '# Layers', '# Parameters', 'Testing Size', 'Accuracy Loss (%)', 
           'Avg Memory Usage (MB)', 'Std Memory Usage', 'Avg Proving Time (s)', 'Std Proving Time']

# Check if the CSV file exists
if not os.path.isfile(csv_path):
    # Create a DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    # Save the DataFrame as a CSV file
    df.to_csv(csv_path, index=False)
else:
    print(f"File '{csv_path}' already exists.")

df = pd.read_csv(csv_path)

File '../../benchmarks/benchmark_results.csv' already exists.


In [8]:
config = {} # Make a shared file for saving all the configs

## Benchmark for 196_25_10 DNN Model

In [9]:
# TODO: use json file for model (results, config etc) management
arch_folder = "input-dense-dense/"
layers = [196,25,10]
model_name = "_".join([str(x) for x in layers])
model_path = "../../models/"

state_dict = torch.load(model_path + arch_folder+ model_name + ".pth")
list_vars = state_dict


In [11]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(layers[0], layers[1])  # Flatten 
        self.fc2 = nn.Linear(layers[1], layers[2])  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim = 1)
    
model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt_downsampled)
    predicted_labels = predictions.argmax(dim=1)

predicted_labels = predicted_labels.tolist()

In [14]:
# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_images_pt_downsampled, test_labels_pt)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')

Accuracy of the PyTorch model on the test images: 95.41000000%


### Convert Model

In [15]:
folder = "./tmp/"

# Create the directory 'tmp' in the current working directory
os.makedirs(folder, exist_ok=True)

In [16]:
fname_out = "./bin/" + arch_folder + "ggml-model-" + model_name + ".bin"
pack_fmt = "!i"

os.makedirs("./bin/" + arch_folder, exist_ok=True)

fout = open(fname_out, "w+b")
fout.write(struct.pack(pack_fmt, 0x67676d6c)) # magic: ggml in hex

for name in list_vars.keys():
    data = list_vars[name].squeeze().numpy()
    print("Processing variable: " + name + " with shape: ", data.shape) 
    n_dims = len(data.shape)
   
    fout.write(struct.pack(pack_fmt, n_dims))
    
    data = data.astype(np.float32)
    for i in range(n_dims):
        fout.write(struct.pack(pack_fmt, data.shape[n_dims - 1 - i]))

    # data
    data = data.astype(">f4")
    data.tofile(fout)

fout.close()

Processing variable: fc1.weight with shape:  (25, 196)
Processing variable: fc1.bias with shape:  (25,)
Processing variable: fc2.weight with shape:  (10, 25)
Processing variable: fc2.bias with shape:  (10,)


In [ ]:
vm_file = "./bin/vm"
program = "./bin/mlgo_196_IDD.bin"
model_in_path = fname_out

### Test

In [17]:
ind = 89
img_out_path = folder + str(ind)
save_img_to_file(test_images_pt_downsampled[ind], img_out_path)

In [18]:
command = [f"{vm_file}", f"--basedir={folder}",
                f"--program={program}", f"--model={model_in_path}", 
                f"--data={img_out_path}", "--mipsVMCompatible"]

In [19]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Get the process ID
pid = process.pid
print(f"Process ID: {pid}")

# Start memory monitoring in a separate thread
monitor_thread = threading.Thread(target=monitor_memory, args=(pid,))
monitor_thread.start()

# Wait for the process to complete and capture output
stdout, stderr = process.communicate()

# Wait for the monitoring thread to finish
monitor_thread.join()

print (f"opml outputs predicted class:{stderr[-2]} where original model pred:{predicted_labels[ind]}")

Process ID: 1751875
opml outputs predicted class:1 where original model pred:1


In [20]:
# Example usage
digits, error = load_img_from_file(img_out_path,show=True)

--------------
--------------
--------------
--------------
--------&-----
--------------
-------&------
------&&------
-------&------
------&-------
------&-------
--------------
--------------
--------------



### Benchmark

In [ ]:
test_size = 5000
loss, mem_usage, time_cost = benchmark(test_images_pt[:test_size], predicted_labels, './tmp/',model_in_path, vm_file=vm_file, program=program)

### Save Results

In [ ]:
new_row = {
    'Framework': ['opml (pytorch)'],
    'Architecture': ['Input-Dense-Dense (196x25x10'],
    '# Layers': [3],
    '# Parameters': [5185],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss/test_size * 100],
    'Avg Memory Usage (MB)': [sum(mem_usage) / len(mem_usage)],
    'Std Memory Usage': [pd.Series(mem_usage).std()],
    'Avg Proving Time (s)': [sum(time_cost) / len(time_cost)],
    'Std Proving Time': [pd.Series(time_cost).std()]
}

new_row_df = pd.DataFrame(new_row)


In [30]:
df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (784 * 56 * 10),3,44543,250,0.40,88.998094,2.285579,3.655122,0.440126
1,opml (pytorch),"Input-Dense-Dense (784 * 56 * 10, w/ relu)",3,44543,1000,20.90,89.122078,2.247846,3.664727,0.433921
2,opml (pytorch),Input-Dense-Dense (784 * 56 * 10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
3,opml (pytorch),Input-Dense-Dense (196 * 25 * 10,3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357


## Benchmark for 784_56_10 DNN Model

In [24]:
# TODO: use json file for model (results, config etc) management
arch_folder = "input-dense-dense/"
layers = [784,56,10]
model_name = "_".join([str(x) for x in layers])
model_path = "../../models/"

state_dict = torch.load(model_path + arch_folder+ model_name + ".pth")
list_vars = state_dict


In [26]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(layers[0], layers[1])  # Flatten 
        self.fc2 = nn.Linear(layers[1], layers[2])  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim = 1)
    
model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt)
    predicted_labels = predictions.argmax(dim=1)

predicted_labels = predicted_labels.tolist()

In [27]:
# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_images_pt, test_labels_pt)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')

Accuracy of the PyTorch model on the test images: 97.40000000%


In [28]:
fname_out = "./bin/" + arch_folder + "ggml-model-" + model_name + ".bin"
pack_fmt = "!i"

os.makedirs("./bin/" + arch_folder, exist_ok=True)

fout = open(fname_out, "w+b")
fout.write(struct.pack(pack_fmt, 0x67676d6c)) # magic: ggml in hex

for name in list_vars.keys():
    data = list_vars[name].squeeze().numpy()
    print("Processing variable: " + name + " with shape: ", data.shape) 
    n_dims = len(data.shape)
   
    fout.write(struct.pack(pack_fmt, n_dims))
    
    data = data.astype(np.float32)
    for i in range(n_dims):
        fout.write(struct.pack(pack_fmt, data.shape[n_dims - 1 - i]))

    # data
    data = data.astype(">f4")
    data.tofile(fout)

fout.close()

Processing variable: fc1.weight with shape:  (56, 784)
Processing variable: fc1.bias with shape:  (56,)
Processing variable: fc2.weight with shape:  (10, 56)
Processing variable: fc2.bias with shape:  (10,)


In [ ]:
vm_file = "./bin/vm"
program = "./bin/mlgo_784_IDD.bin"
model_in_path = fname_out

### Test

In [32]:
ind = 89
img_out_path = folder + str(ind)
save_img_to_file(test_images_pt[ind], img_out_path)

In [33]:
command = [f"{vm_file}", f"--basedir={folder}",
                f"--program={program}", f"--model={model_in_path}", 
                f"--data={img_out_path}", "--mipsVMCompatible"]

In [34]:
subprocess.run(command)

modelSize:  178164
rawSize:  [0 2 183 244]


Start MIPS MNIST
start MIPS_InputProcess
buf len:  784
____________________________
____________________________
____________________________
____________________________
_________________*__________
_________________*__________
_________________*__________
_________________*__________
________________**__________
_______________***__________
_______________**___________
_______________*____________
______________**____________
_____________***____________
____________****____________
_____________**_____________
_____________***____________
____________***_____________
____________***_____________
____________***_____________
___________****_____________
___________***______________
___________***______________
____________________________
____________________________
____________________________
____________________________
____________________________

start MIPS_mnist_model_load
model_bytes len:  178164
magic: 67676d6c
reading fc1
n_dims:  2
ne_weight:  [784 56]
index:  175632
read

reach the final state, total step: 31931367, target: -1
writing ./tmp//checkpoint_31931367.json len 7582652 with root 0x88b8d7cae8f35d599766f1ae2a6be8cef88e3ffe698be6220fbb8881eb4bca79
lastStep:  31931367
writing ./tmp//checkpoint_final.json len 7582652 with root 0x88b8d7cae8f35d599766f1ae2a6be8cef88e3ffe698be6220fbb8881eb4bca79
PC: 5ead0000


CompletedProcess(args=['./bin/vm', '--basedir=./tmp/', '--program=./bin/mlgo_784_IDD.bin', '--model=./bin/input-dense-dense/ggml-model-784_56_10.bin', '--data=./tmp/89', '--mipsVMCompatible'], returncode=0)

In [35]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Get the process ID
pid = process.pid
print(f"Process ID: {pid}")

# Start memory monitoring in a separate thread
monitor_thread = threading.Thread(target=monitor_memory, args=(pid,))
monitor_thread.start()

# Wait for the process to complete and capture output
stdout, stderr = process.communicate()

# Wait for the monitoring thread to finish
monitor_thread.join()

print (f"opml outputs predicted class:{stderr[-2]} where original model pred:{predicted_labels[ind]}")

Process ID: 1752142
opml outputs predicted class:1 where original model pred:1


### Benchmark

In [36]:
test_size = 5
loss, mem_usage, time_cost = benchmark(test_images_pt[:test_size], predicted_labels, './tmp/',model_in_path, vm_file=vm_file, program=program)

Process for image 0
Process for image 1
Process for image 2
Process for image 3
Process for image 4
Total time: 17.08100390434265


In [ ]:
new_row = {
    'Framework': ['opml (pytorch)'],
    'Architecture': ['Input-Dense-Dense (784x56x10'],
    '# Layers': [3],
    '# Parameters': [44543],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss/test_size*100],
    'Avg Memory Usage (MB)': [sum(mem_usage) / len(mem_usage)],
    'Std Memory Usage': [pd.Series(mem_usage).std()],
    'Avg Proving Time (s)': [sum(time_cost) / len(time_cost)],
    'Std Proving Time': [pd.Series(time_cost).std()]
}

new_row_df = pd.DataFrame(new_row)


In [ ]:
df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

## Benchmark for 196_24_14_10 DNN Model

In [55]:
# TODO: use json file for model (results, config etc) management
arch_folder = "input-dense-dense-dense/"
layers = [196,24,14,10]
model_name = "_".join([str(x) for x in layers])
model_path = "../../models/"

state_dict = torch.load(model_path + arch_folder+ model_name + ".pth")
list_vars = state_dict


In [56]:
# 196_24_14_10
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(layers[0], layers[1])  # Flatten 
        self.fc2 = nn.Linear(layers[1], layers[2])
        self.fc3 = nn.Linear(layers[2], num_classes)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim = 1)
    
model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt_downsampled)
    predicted_labels = predictions.argmax(dim=1)

predicted_labels = predicted_labels.tolist()

In [57]:
# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_images_pt_downsampled, test_labels_pt)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')

Accuracy of the PyTorch model on the test images: 95.56000000%


### Convert Model

In [58]:
tmp_folder = './tmp/'
os.makedirs(tmp_folder, exist_ok=True)

In [59]:
fname_out = "./bin/" + arch_folder + "ggml-model-" + model_name + ".bin"
pack_fmt = "!i"

os.makedirs("./bin/" + arch_folder, exist_ok=True)

fout = open(fname_out, "w+b")
fout.write(struct.pack(pack_fmt, 0x67676d6c)) # magic: ggml in hex

for name in list_vars.keys():
    data = list_vars[name].squeeze().numpy()
    print("Processing variable: " + name + " with shape: ", data.shape) 
    n_dims = len(data.shape)
   
    fout.write(struct.pack(pack_fmt, n_dims))
    
    data = data.astype(np.float32)
    for i in range(n_dims):
        fout.write(struct.pack(pack_fmt, data.shape[n_dims - 1 - i]))

    # data
    data = data.astype(">f4")
    data.tofile(fout)

fout.close()

Processing variable: fc1.weight with shape:  (24, 196)
Processing variable: fc1.bias with shape:  (24,)
Processing variable: fc2.weight with shape:  (14, 24)
Processing variable: fc2.bias with shape:  (14,)
Processing variable: fc3.weight with shape:  (10, 14)
Processing variable: fc3.bias with shape:  (10,)


In [60]:
vm_file = "./bin/vm"
program = "./bin/mlgo_196_IDDD.bin"
model_in_path = fname_out

### Test

In [61]:
ind = 10
img_out_path = folder + str(ind)
save_img_to_file(test_images_pt_downsampled[ind], img_out_path)

In [62]:
command = [f"{vm_file}", f"--basedir={folder}",
                f"--program={program}", f"--model={model_in_path}", 
                f"--data={img_out_path}", "--mipsVMCompatible"]

In [ ]:
subprocess.run(command)

In [64]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Get the process ID
pid = process.pid
print(f"Process ID: {pid}")

# Start memory monitoring in a separate thread
monitor_thread = threading.Thread(target=monitor_memory, args=(pid,))
monitor_thread.start()

# Wait for the process to complete and capture output
stdout, stderr = process.communicate()

# Wait for the monitoring thread to finish
monitor_thread.join()

print (f"opml outputs predicted class:{stderr[-2]} where original model pred:{predicted_labels[ind]}")

Process ID: 1885929
opml outputs predicted class:0 where original model pred:0


### Benchmark

In [72]:
test_size = 5000
loss, mem_usage, time_cost = benchmark(test_images_pt_downsampled[:test_size], predicted_labels, './tmp/',model_in_path, vm_file=vm_file, program=program)

Process for image 0
Process for image 1
Process for image 2
Process for image 3
Process for image 4
Process for image 5
Process for image 6
Process for image 7
Process for image 8
Process for image 9
Process for image 10
Process for image 11
Process for image 12
Process for image 13
Process for image 14
Process for image 15
Process for image 16
Process for image 17
Process for image 18
Process for image 19
Process for image 20
Process for image 21
Process for image 22
Process for image 23
Process for image 24
Process for image 25
Process for image 26
Process for image 27
Process for image 28
Process for image 29
Process for image 30
Process for image 31
Process for image 32
Process for image 33
Process for image 34
Process for image 35
Process for image 36
Process for image 37
Process for image 38
Process for image 39
Process for image 40
Process for image 41
Process for image 42
Process for image 43
Process for image 44
Process for image 45
Process for image 46
Process for image 47
Pr

In [73]:
new_row = {
    'Framework': ['opml (pytorch)'],
    'Architecture': ['Input-Dense-Dense-Dense (196x24x14x10'],
    '# Layers': [4],
    '# Parameters': [5228],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss/test_size * 100],
    'Avg Memory Usage (MB)': [sum(mem_usage) / len(mem_usage)],
    'Std Memory Usage': [pd.Series(mem_usage).std()],
    'Avg Proving Time (s)': [sum(time_cost) / len(time_cost)],
    'Std Proving Time': [pd.Series(time_cost).std()]
}

new_row_df = pd.DataFrame(new_row)

In [75]:
df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (196x25x10),3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
1,opml (pytorch),Input-Dense-Dense (784x56x10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
2,circomlib-ml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491
3,circomlib-ml (tensorflow),Input-Dense-Dense (784x56x10,3,44543,2500,0.04,2328.322411,34.436486,2.351284,0.147113
4,zkml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,1000,0.10,2334.295629,12.335301,21.263430,0.294183
5,zkml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,1000,0.00,2357.363156,11.750076,22.012543,0.317096
6,opml (pytorch),Input-Dense-Dense-Dense (196x24x14x10,4,5228,5000,3.42,74.450789,1.491530,0.829767,0.085084
